In [31]:
from utils.BackTest import Engine, Strategy
import pandas as pd
import xgboost as xgb
import numpy as np
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
import optuna
import optuna.visualization as vis
import os

c:\Users\leoqi\miniforge3\envs\Finance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
df = pd.read_csv(r'../data/processed/merged_df_with_signals.csv')

In [33]:
df

,datetime,reddit_smoothed_sentiment_weibull_week,news_smoothed_sentiment_weibull_week,tele_smoothed_sentiment_weibull_week,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,...,reddit_sentiment_log,news_sentiment_log,tele_sentiment_log,reddit_Signal,news_Signal,tele_Signal,typical_price,DATE,VWAP,VWAP_Signal
0,2024-01-27 13:00:00,0.000000,-857.923901,6.872941,41750.00,41806.55,41725.21,41742.44,35569,10171,...,0.000000,-6.755482,2.060767,0.0,-1.0,1.0,41758.066667,2024-01-27,41758.066667,-1
1,2024-01-27 14:00:00,0.027001,-867.694664,6.872941,41742.44,41818.39,41731.01,41766.00,27238,12281,...,0.026643,-6.766796,2.060767,1.0,-1.0,1.0,41771.800000,2024-01-27,41765.490372,1
2,2024-01-27 15:00:00,0.164191,-875.992357,6.872941,41766.00,41894.87,41733.60,41814.00,32410,14004,...,0.152026,-6.776304,2.060767,1.0,-1.0,1.0,41814.156667,2024-01-27,41784.423590,1
3,2024-01-27 16:00:00,0.300527,-882.407146,6.892280,41814.00,41895.74,41791.22,41810.18,30278,13664,...,0.262769,-6.783594,2.063227,1.0,-1.0,1.0,41832.380000,2024-01-27,41798.313570,1
4,2024-01-27 17:00:00,0.435303,-886.430380,6.892280,41810.18,41845.34,41774.07,41779.28,24681,9330,...,0.361376,-6.788139,2.063227,1.0,-1.0,1.0,41799.563333,2024-01-27,41798.533122,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,2025-03-26 08:00:00,-0.487101,-43.013907,12.500145,88140.45,88200.00,87886.10,87959.73,59816,30655,...,-0.396829,-3.780636,2.601147,-1.0,-1.0,1.0,88015.276667,2025-03-26,87642.510178,1
10172,2025-03-26 09:00:00,-0.456887,-39.331997,12.500145,87959.73,88056.61,87837.49,87939.99,43067,23857,...,-0.376302,-3.692921,2.601147,-1.0,-1.0,1.0,87944.696667,2025-03-26,87661.828105,1
10173,2025-03-26 10:00:00,-0.427685,-37.038648,12.500145,87939.99,88259.81,87939.99,88208.47,55584,29793,...,-0.356054,-3.634124,2.601147,-1.0,-1.0,1.0,88136.090000,2025-03-26,87701.167882,1
10174,2025-03-26 11:00:00,-0.399548,-37.530942,12.500145,88208.47,88242.00,87923.91,87954.96,67105,30529,...,-0.336149,-3.647040,2.601147,-1.0,-1.0,1.0,88040.290000,2025-03-26,87742.875495,1


In [34]:
base_price_data = ['datetime','Close','Open','High','Low','VOLUME']

technical_indicators = [
    # Trend Indicators
    'EMA_8', 'EMA_13', 'EMA_21', 'EMA_Signal', 'EMA_short', 'EMA_long',
    'SMA20', 'SMA_50', 'SMA_200', 'GoldenCross_Signal',

    # Momentum Indicators
    'MACD', 'Signal_Line', 'MACD_Signal', 'MACD_Hist',
    'RSI_14', 'BB_RSI_Signal', 'RSI_Signal',
    '%K', '%D', 'Stochastic_Signal',

    # Volatility Indicators
    'STD20', 'Upper_BB', 'Lower_BB', 'ATR', 'TR',

    # Volume-Based Indicators
    'OBV', 'OBV_EMA20', 'OBV_Signal', 'VWAP_Signal'
]

sentiment_indicators = [
    'reddit_sentiment_log',
    'tele_sentiment_log',
    'news_sentiment_log',
]

# Create a new DataFrame with the selected columns
selected_columns = base_price_data + technical_indicators + sentiment_indicators
df_selected = df[selected_columns]

In [35]:
working_frame = df_selected.copy().set_index(pd.to_datetime(df_selected['datetime'])).drop(columns=['datetime'])
working_frame

,Close,Open,High,Low,VOLUME,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,...,Lower_BB,ATR,TR,OBV,OBV_EMA20,OBV_Signal,VWAP_Signal,reddit_sentiment_log,tele_sentiment_log,news_sentiment_log
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-27 13:00:00,41742.44,41750.00,41806.55,41725.21,419.38468,41729.550796,41714.525206,41617.761701,1,41720.541302,...,41524.488454,175.710000,81.34,-419.38468,-65.712408,0,-1,0.000000,2.060767,-6.755482
2024-01-27 14:00:00,41766.00,41742.44,41818.39,41731.01,493.43458,41737.650619,41721.878748,41631.237910,1,41727.534948,...,41541.267296,140.500000,87.38,493.43458,-12.460314,1,1,0.026643,2.060767,-6.766796
2024-01-27 15:00:00,41814.00,41766.00,41894.87,41733.60,581.25856,41754.617148,41735.038927,41647.852645,1,41740.837263,...,41561.147977,109.318333,161.27,581.25856,44.084341,0,1,0.152026,2.060767,-6.776304
2024-01-27 16:00:00,41810.18,41814.00,41895.74,41791.22,609.18361,41766.964449,41745.773366,41662.609678,1,41751.505377,...,41571.210298,101.921667,104.52,-609.18361,-18.131655,-1,1,0.262769,2.063227,-6.783594
2024-01-27 17:00:00,41779.28,41810.18,41845.34,41774.07,448.23397,41769.701238,41750.560028,41673.216071,1,41755.778396,...,41588.600128,99.801667,71.27,-448.23397,-59.093780,0,-1,0.361376,2.063227,-6.788139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-26 08:00:00,87959.73,88140.45,88200.00,87886.10,525.11331,87756.884904,87689.841782,87613.211597,1,87700.669674,...,87158.682498,418.066667,313.90,-525.11331,59.670458,-1,1,-0.396829,2.601147,-3.780636
2025-03-26 09:00:00,87939.99,87959.73,88056.61,87837.49,393.09750,87797.574925,87725.577241,87642.918724,1,87737.488186,...,87170.539414,392.118333,219.12,-393.09750,16.549700,0,1,-0.376302,2.601147,-3.692921
2025-03-26 10:00:00,88208.47,87939.99,88259.81,87939.99,556.20541,87888.884942,87794.561921,87694.332477,1,87809.946926,...,87154.234720,378.148333,319.82,556.20541,67.945482,1,1,-0.356054,2.601147,-3.634124


In [43]:
loaded_model = xgb.XGBRegressor()
loaded_model.load_model('xgboost_model.json')

In [44]:
loaded_final_model = xgb.XGBRegressor()
loaded_final_model.load_model('xgboost_model_final.json')

In [45]:
labels = pd.read_parquet(f'../data/processed/labels_btc.parquet')
inputs = np.load(f'../data/processed/inputs_btc.npy')

In [46]:
class XGBoostStrategy(Strategy):
    def __init__(self,model,bet_size = 0.1,initial_data=None):
        super().__init__()
        self.model = model 
        self.initial_data = initial_data
        self.bet_size = bet_size  # 10% of cash

    def on_bar(self):

        temp_df = pd.concat([self.initial_data, self.data])

        features = self.sequencing_data(temp_df, window_size=7)

        # ###TESTING INPUTS###
        # print("Current index:", self.current_idx)

        # idx = labels[labels['inference_date'] == self.current_idx].index
        # arr = inputs[idx]
        # are_close = np.allclose(arr, features, atol=1e-5)

        # print("Arrays are close:", are_close)

        # if self.current_idx == pd.to_datetime('2025-02-28 00:00:00'):
        #     print(features)

        predictions = self.model.predict(features)
        signal = self.determine_signal(predictions)
        if signal == 1:
            self.buy(
                "btc",
               size = round((self.bet_size * self.cash) / self.close(), 8)
            )
        elif signal == -1:
            self.sell(
                "btc",
                size = round((self.bet_size  * self.cash) / self.close(), 8)
            )
        else:
            pass

    def determine_signal(self, predictions):
        """Determine buy/sell/hold signal based on predictions."""
        future_forecasts = predictions.flatten().tolist()  # Convert to list if needed
        close = self.close()

        above_close = sum(forecast > close for forecast in future_forecasts)
        below_close = sum(forecast < close for forecast in future_forecasts)

        if above_close >= 3:
            return 1  # Buy
        elif below_close >= 3:
            return -1  # Sell
        else:
           return 0  # Hold
        
    def sequencing_data(self, df, window_size=7):
        start_time = self.current_idx - pd.Timedelta(hours=window_size)
        past_data = df.loc[start_time:self.current_idx - pd.Timedelta(hours=1)]

        # Collect past feature lists
        past_features = [
            past_data['High'].tolist(),
            past_data['Low'].tolist(),
            past_data['Open'].tolist(),
            past_data['VOLUME'].tolist()
        ]

        # Technical Indicators
        indicators = [
            'EMA_8', 'EMA_13', 'EMA_21', 'EMA_Signal', 'EMA_short', 'EMA_long',
            'SMA20', 'SMA_50', 'SMA_200', 'GoldenCross_Signal',
            'MACD', 'Signal_Line', 'MACD_Signal', 'MACD_Hist',
            'RSI_14', 'BB_RSI_Signal', 'RSI_Signal',
            '%K', '%D', 'Stochastic_Signal',
            'STD20', 'Upper_BB', 'Lower_BB', 'ATR', 'TR',
            'OBV', 'OBV_EMA20', 'OBV_Signal', 'VWAP_Signal'
        ]
        
        for indicator in indicators:
            past_features.append(past_data[indicator].tolist())

        # Sentiment indicators
        sentiments = ['reddit_sentiment_log', 'tele_sentiment_log', 'news_sentiment_log']
        
        for sentiment in sentiments:
            past_features.append(past_data[sentiment].tolist())

        # Convert to 1D NumPy array and reshape to (1, 252)
        past_features_array = np.concatenate(past_features).reshape(1, -1)
        
        return past_features_array  # Shape: (1, 252)

In [47]:
initial_val_frame = working_frame[(working_frame.index >= pd.to_datetime('2025-01-01') - pd.Timedelta(hours=7)) & (working_frame.index < '2025-01-01')]
validate_frame = working_frame[(working_frame.index >= '2025-01-01') & (working_frame.index < '2025-03-01')]

initial_test_frame = working_frame[(working_frame.index >= pd.to_datetime('2025-03-01') - pd.Timedelta(hours=7)) & (working_frame.index < '2025-03-01')]
test_frame = working_frame[working_frame.index >= '2025-03-01']

In [48]:
validate_frame

,Close,Open,High,Low,VOLUME,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,...,Lower_BB,ATR,TR,OBV,OBV_EMA20,OBV_Signal,VWAP_Signal,reddit_sentiment_log,tele_sentiment_log,news_sentiment_log
datetime,,,,,,,,,,,,,,,,,,,,,
2025-01-01 00:00:00,94401.14,93576.00,94509.42,93489.03,755.99010,94011.893818,94029.638251,93958.906655,0,94032.973503,...,92381.690536,622.563333,1020.39,755.99010,103.811170,0,1,0.450214,2.029984,-6.587146
2025-01-01 01:00:00,93607.74,94401.14,94408.72,93578.77,586.53456,93922.081858,93969.367072,93926.982413,-1,93967.552964,...,92563.555167,664.478333,829.95,-586.53456,38.063957,-1,-1,0.375333,2.029984,-6.607708
2025-01-01 02:00:00,94098.91,93607.74,94105.12,93594.56,276.78045,93961.377001,93987.873205,93942.612194,0,93987.761739,...,92755.139100,620.850000,510.56,276.78045,60.798861,1,1,0.295866,2.029984,-6.627062
2025-01-01 03:00:00,93838.04,94098.91,94098.91,93728.22,220.99302,93933.968779,93966.468461,93933.105631,0,93964.727625,...,92934.645329,606.051667,370.69,-220.99302,33.961539,-1,-1,0.211558,2.029984,-6.644706
2025-01-01 04:00:00,93553.91,93838.04,93838.04,93500.00,279.46909,93849.511272,93907.531538,93898.633301,-1,93901.524914,...,92885.780670,575.041667,338.04,-279.46909,4.111003,0,-1,0.122122,2.029984,-6.659001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28 19:00:00,84449.99,84440.97,84810.35,84209.84,1656.89446,83302.202137,82785.136766,82669.910295,1,82850.241395,...,77705.107174,1456.295000,600.51,1656.89446,484.835909,0,1,-1.221349,0.913890,-8.396762
2025-02-28 20:00:00,84191.27,84449.99,84795.03,83600.00,2215.50549,83499.772773,82986.012942,82808.215723,1,83056.553488,...,77711.558133,1409.260000,1195.03,-2215.50549,227.660538,-1,1,-1.246760,0.913890,-8.393514
2025-02-28 21:00:00,84258.37,84191.27,84596.00,84000.17,862.30163,83668.349935,83167.778236,82940.047930,1,83241.448336,...,77655.745657,1103.960000,595.83,862.30163,288.102547,1,1,-1.269582,0.919505,-8.389415


In [49]:
initial_val_frame

,Close,Open,High,Low,VOLUME,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,...,Lower_BB,ATR,TR,OBV,OBV_EMA20,OBV_Signal,VWAP_Signal,reddit_sentiment_log,tele_sentiment_log,news_sentiment_log
datetime,,,,,,,,,,,,,,,,,,,,,
2024-12-31 17:00:00,94120.94,94868.42,95047.71,93189.59,1332.40373,94617.102917,94342.895025,94052.159809,1,94394.046588,...,91330.978189,1174.988333,1858.12,-1332.40373,133.655000,0,-1,0.668578,1.968432,-6.382596
2024-12-31 18:00:00,93875.68,94120.94,94416.72,93779.80,816.81017,94452.342269,94276.150021,94036.116190,1,94314.297882,...,91418.780827,1160.111667,636.92,-816.81017,43.134507,0,-1,0.644116,1.968432,-6.413587
2024-12-31 19:00:00,94166.88,93875.68,94290.91,93712.45,462.79342,94388.906209,94260.540018,94048.003809,1,94291.618208,...,91518.783895,1001.461667,578.46,462.79342,83.102023,1,1,0.618698,2.029984,-6.449630
2024-12-31 20:00:00,93564.04,94166.88,94222.50,93450.17,733.04147,94205.602607,94161.040016,94004.007099,1,94179.683099,...,91657.553926,983.908333,772.33,-733.04147,5.374071,-1,-1,0.592473,2.029984,-6.482636
2024-12-31 21:00:00,93899.68,93564.04,93964.15,93504.67,337.52715,94137.619805,94123.702871,93994.522817,1,94136.605699,...,91830.596274,881.571667,459.48,337.52715,37.007698,1,-1,0.565590,2.029984,-6.514657
2024-12-31 22:00:00,93488.84,93899.68,93899.68,93375.58,315.53272,93993.446515,94033.008175,93948.551652,0,94036.949438,...,91979.914579,804.901667,524.10,-315.53272,3.432420,-1,-1,0.538200,2.029984,-6.543260
2024-12-31 23:00:00,93576.00,93488.84,93756.00,93375.38,336.57995,93900.680623,93967.721293,93914.683320,-1,93966.034140,...,92141.877951,558.651667,380.62,336.57995,35.160756,1,-1,0.510451,2.029984,-6.566459


In [52]:
# Store all trials' results
all_trials = []

def objective(trial):
    # Optimize bet size between 1% and 50% of capital
    bet_size = trial.suggest_float("bet_size", 0.01, 0.5)  

    # Run backtest with the given bet size
    e = Engine()
    e.add_data(validate_frame)
    e.add_strategy(XGBoostStrategy(model=loaded_model, bet_size=bet_size,initial_data= initial_val_frame))  # Pass optimized bet size

    stats = e.run()

    # Save trial results
    trial_result = {
        "trial": trial.number,
        "sharpe": stats.get("Strategy Sharpe Ratio", np.nan),
        "return": stats.get("Total Return (%)", np.nan),
        "drawdown": stats.get("Strategy Max Drawdown (%)", np.nan),
        "bet_size": bet_size,
    }
    all_trials.append(trial_result)

    return trial_result["sharpe"]  # Optimize for Sharpe Ratio

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Save results
results_df = pd.DataFrame(all_trials)

print("Best Sharpe:", study.best_value)
print("Best Bet Size:", study.best_params["bet_size"])

# Plot optimization results
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()

[I 2025-04-03 12:20:29,070] A new study created in memory with name: no-name-c8a508ec-6576-413e-ac05-6cab47de3f26
100%|██████████| 1416/1416 [00:03<00:00, 442.58it/s]
[I 2025-04-03 12:20:32,293] Trial 0 finished with value: 23654.68530483821 and parameters: {'bet_size': 0.2360442682896538}. Best is trial 0 with value: 23654.68530483821.


Final Portfolio Value: 1063414.0648738286
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |     963.41
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |   -1124.69
Strategy CAGR (%)                        | 2889547.79
Buy & Hold CAGR (%)                      |     -38.68
Strategy Volatility (%)                  |     122.16
Buy & Hold Volatility (%)                |       8.81
Strategy Sharpe Ratio                    |   23654.69
Buy & Hold Sharpe Ratio                  |      -4.39
Strategy Max Drawdown (%)                |     -75.79
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1261.00
Number of Buys                           |     553.00
Number of Sells                          |     708.00
Win Rate (%)      

100%|██████████| 1416/1416 [00:03<00:00, 431.90it/s]
[I 2025-04-03 12:20:35,584] Trial 1 finished with value: 7386.179649875717 and parameters: {'bet_size': 0.38807585388892774}. Best is trial 0 with value: 23654.68530483821.


Final Portfolio Value: 1700304.3983147815
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |    1600.30
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |   -1859.11
Strategy CAGR (%)                        | 22203646.34
Buy & Hold CAGR (%)                      |     -38.68
Strategy Volatility (%)                  |    3006.11
Buy & Hold Volatility (%)                |       8.81
Strategy Sharpe Ratio                    |    7386.18
Buy & Hold Sharpe Ratio                  |      -4.39
Strategy Max Drawdown (%)                |    -106.23
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1259.00
Number of Buys                           |     551.00
Number of Sells                          |     708.00
Win Rate (%)     

100%|██████████| 1416/1416 [00:03<00:00, 427.86it/s]
[I 2025-04-03 12:20:38,913] Trial 2 finished with value: 35841.187931973545 and parameters: {'bet_size': 0.27351032561626865}. Best is trial 2 with value: 35841.187931973545.


...

Final Portfolio Value: 1527192.8401841992
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |    1427.19
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |    -393.10
Strategy CAGR (%)                        | 13925527.89
Buy & Hold CAGR (%)                      |     -38.68
Strategy Volatility (%)                  |   10098.32
Buy & Hold Volatility (%)                |       8.81
Strategy Sharpe Ratio                    |    1378.99
Buy & Hold Sharpe Ratio                  |      -4.39
Strategy Max Drawdown (%)                |    -100.95
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1259.00
Number of Buys                           |     551.00
Number of Sells                          |     708.00
Win Rate (%)     

100%|██████████| 1416/1416 [00:04<00:00, 347.80it/s]
[I 2025-04-03 12:26:53,129] Trial 98 finished with value: 20849.567610115126 and parameters: {'bet_size': 0.35962423961210077}. Best is trial 58 with value: 58127.8481681172.


Final Portfolio Value: 1582978.7721480397
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |    1482.98
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |   -1643.18
Strategy CAGR (%)                        | 16274580.23
Buy & Hold CAGR (%)                      |     -38.68
Strategy Volatility (%)                  |     780.57
Buy & Hold Volatility (%)                |       8.81
Strategy Sharpe Ratio                    |   20849.57
Buy & Hold Sharpe Ratio                  |      -4.39
Strategy Max Drawdown (%)                |    -102.72
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1259.00
Number of Buys                           |     551.00
Number of Sells                          |     708.00
Win Rate (%)     

100%|██████████| 1416/1416 [00:03<00:00, 369.96it/s]
[I 2025-04-03 12:26:56,982] Trial 99 finished with value: 53297.53638510913 and parameters: {'bet_size': 0.3192644479551254}. Best is trial 58 with value: 58127.8481681172.


Final Portfolio Value: 1402888.4480098598
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |    1302.89
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |   -1397.14
Strategy CAGR (%)                        | 9629711.29
Buy & Hold CAGR (%)                      |     -38.68
Strategy Volatility (%)                  |     180.68
Buy & Hold Volatility (%)                |       8.81
Strategy Sharpe Ratio                    |   53297.54
Buy & Hold Sharpe Ratio                  |      -4.39
Strategy Max Drawdown (%)                |     -88.48
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1260.00
Number of Buys                           |     552.00
Number of Sells                          |     708.00
Win Rate (%)      

In [54]:
best_params = study.best_params
best_params

{'bet_size': 0.3320661600396659}

In [58]:
e = Engine(asset_type= 'cryptocurrencies')
e.add_data(validate_frame)  #
e.add_strategy(XGBoostStrategy(
    model=loaded_model,
    bet_size=best_params["bet_size"],
    initial_data=initial_val_frame,  # Pass initial data for sequencing
))
val_stats = e.run()
fig = e.plot()

100%|██████████| 1416/1416 [00:03<00:00, 431.96it/s]


Final Portfolio Value: 1455131.0400583027
Initial Portfolio Value: 100000
Buy & Hold Final Value: 89960.37923179021
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |    1355.13
Buy-and-Hold Total Return (%)            |     -10.04
Average Exposure to Asset (%)            |   -1443.27
Strategy CAGR (%)                        | 2080957572.77
Buy & Hold CAGR (%)                      |     -50.76
Strategy Volatility (%)                  |     233.70
Buy & Hold Volatility (%)                |      10.61
Strategy Sharpe Ratio                    | 8904222.67
Buy & Hold Sharpe Ratio                  |      -4.79
Strategy Max Drawdown (%)                |     -90.12
Buy & Hold Max Drawdown (%)              |     -27.09
Number of Trades                         |    1260.00
Number of Buys                           |     552.00
Number of Sells                          |     708.00
Win Rate (%)   

In [61]:
#GOOD FROM 1% to 1.4% transaction cost. >1.5% sharpe ratio is worse than buy and hold

e = Engine(initial_cash = 100_000,transaction_cost= 0.001, asset_type='cryptocurrencies')
e.add_data(test_frame)
e.add_strategy(XGBoostStrategy(loaded_final_model,bet_size=best_params["bet_size"],initial_data= initial_test_frame)) # Pass both df_filtered and strategy_dict
stats = e.run()


100%|██████████| 613/613 [00:01<00:00, 427.60it/s]

Final Portfolio Value: 119738.28623638046
Initial Portfolio Value: 100000
Buy & Hold Final Value: 103897.68086755012
Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      19.74
Buy-and-Hold Total Return (%)            |       3.90
Average Exposure to Asset (%)            |    -233.48
Strategy CAGR (%)                        |    1287.40
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      53.81
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      23.92
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -46.40
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     403.00
Number of Buys                           |     177.00
Number of Sells                          |     226.00
Win Rate (%)     

In [62]:
e.strategy.trades

[<Trade: 2025-03-01 01:00:00 btc buy 0.39598664@83857.92 Role:entry PnL:0>,
 <Trade: 2025-03-01 02:00:00 btc sell 0.39224923@84656.93 Role:exit PnL:313.41105726229796>,
 <Trade: 2025-03-01 03:00:00 btc sell 0.0037374099999999966@85240.46 Role:exit PnL:5.167118821400026>,
 <Trade: 2025-03-01 04:00:00 btc sell 0.3891514@85330.84 Role:entry PnL:0>,
 <Trade: 2025-03-01 05:00:00 btc sell 0.38527221@86190.01 Role:entry PnL:0>,
 <Trade: 2025-03-01 06:00:00 btc buy 0.38798129@85588.19 Role:cover PnL:-99.84698498150226>,
 <Trade: 2025-03-01 07:00:00 btc buy 0.38644231999999995@85075.62 Role:cover PnL:98.62780891040043>,
 <Trade: 2025-03-01 08:00:00 btc buy 0.39186402@84740.15 Role:entry PnL:0>,
 <Trade: 2025-03-01 09:00:00 btc sell 0.38927672@85303.37 Role:exit PnL:219.24843423840048>,
 <Trade: 2025-03-01 10:00:00 btc sell 0.002587299999999959@85380.0 Role:exit PnL:1.6554839049999888>,
 <Trade: 2025-03-01 11:00:00 btc buy 0.39112617@84900.01 Role:entry PnL:0>,
 <Trade: 2025-03-01 12:00:00 btc b

In [63]:
fig = e.plot()

In [30]:
# Define the parameter ranges
initial_cash_values = range(100_000, 750_000, 50_000)  # 100k to 700k in 50k steps
transaction_cost_values = [i/10000 for i in range(10, 51,5)]  # 0.001 to 0.002 (0.1% to 0.5%)

# Initialize results storage
results = []

# Run backtests for all combinations
for initial_cash in tqdm(initial_cash_values, desc="Initial Cash"):
    row = {}
    for transaction_cost in tqdm(transaction_cost_values, desc=f"Transaction Costs for ${initial_cash:,}", leave=False):
        # Create and run engine
        e = Engine(
            initial_cash=initial_cash,
            transaction_cost=transaction_cost,
            asset_type='cryptocurrencies'
        )
        e.add_data(test_frame)
        e.add_strategy(XGBoostStrategy(loaded_model, initial_data=initial_test_frame))
        stats = e.run()
        
        # Store results
        key = f"{transaction_cost*100:.2f}%"
        row[key] = (
            round(stats['Strategy Sharpe Ratio'], 2),
            round(stats['Buy & Hold Sharpe Ratio'], 2)
        )
    results.append(row)

# Convert results to DataFrame
df_results = pd.DataFrame(
    results,
    index=[f"${x:,}" for x in initial_cash_values]
)

# Display the results table
print("Sharpe Ratios (Strategy, Buy & Hold) for different parameters:")
print(df_results)


Initial Cash:   0%|          | 0/13 [00:00<?, ?it/s]












100%|██████████| 613/613 [00:01<00:00, 446.04it/s]


Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |       7.97
Buy-and-Hold Total Return (%)            |       4.00
Average Exposure to Asset (%)            |     -41.49
Strategy CAGR (%)                        |     206.24
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      17.44
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      11.82
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -18.04
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     411.00
Number of Buys                           |     185.00
Number of Sells                          |     226.00
Win Rate (%)                             |      47.89
Avg Profit per Trade                     |     -84.29
















100%|██████████| 613/613 [00:01<00:00, 446.46it/s]


Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |       5.91
Buy-and-Hold Total Return (%)            |       3.95
Average Exposure to Asset (%)            |     -42.21
Strategy CAGR (%)                        |     131.32
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      17.58
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |       7.47
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -18.63
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     411.00
Number of Buys                           |     185.00
Number of Sells                          |     226.00
Win Rate (%)                             |      47.89
Avg Profit per Trade                     |     -84.29















100%|██████████| 613/613 [00:01<00:00, 436.99it/s]


...

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      -4.51
Buy-and-Hold Total Return (%)            |       3.69
Average Exposure to Asset (%)            |     -53.22
Strategy CAGR (%)                        |     -49.04
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      18.27
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      -2.68
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -22.61
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     411.00
Number of Buys                           |     185.00
Number of Sells                          |     226.00
Win Rate (%)                             |      47.09
Avg Profit per Trade                     |    -554.68


















100%|██████████| 613/613 [00:01<00:00, 361.63it/s]


Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      -6.57
Buy-and-Hold Total Return (%)            |       3.64
Average Exposure to Asset (%)            |     -54.22
Strategy CAGR (%)                        |     -62.91
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      18.45
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      -3.41
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -23.79
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     411.00
Number of Buys                           |     185.00
Number of Sells                          |     226.00
Win Rate (%)                             |      47.09
Avg Profit per Trade                     |    -554.68


















Initial Cash: 100%|██████████| 13/13 [04:09<00:00, 19.16s/it]

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      -8.62
Buy-and-Hold Total Return (%)            |       3.59
Average Exposure to Asset (%)            |     -55.27
Strategy CAGR (%)                        |     -73.19
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      18.63
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      -3.93
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -24.98
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     411.00
Number of Buys                           |     185.00
Number of Sells                          |     226.00
Win Rate (%)                             |      47.09
Avg Profit per Trade                     |    -554.68
Sharpe Ratios (Strategy, B

In [36]:
transaction_cost_values

[0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004, 0.0045, 0.005]

In [35]:
df_results

,0.10%,0.15%,0.20%,0.25%,0.30%,0.35%,0.40%,0.45%,0.50%
"$100,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$150,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$200,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$250,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$300,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$350,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$400,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$450,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$500,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$550,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"


In [33]:
df_results = df_results.map(lambda x: (float(x[0]), float(x[1])))
def color_cells(val):
    strategy, buy_hold = val
    if strategy > buy_hold:
        return 'background-color: green; color: white'
    else:
        return 'background-color: red; color: white'
    
styled_table = df_results.style.map(color_cells)
styled_table    

,0.10%,0.15%,0.20%,0.25%,0.30%,0.35%,0.40%,0.45%,0.50%
"$100,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$150,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$200,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$250,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$300,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$350,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$400,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$450,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$500,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
"$550,000","(11.82, 6.95)","(7.47, 6.95)","(3.37, 6.95)","(1.52, 6.95)","(-0.32, 6.95)","(-1.68, 6.95)","(-2.68, 6.95)","(-3.41, 6.95)","(-3.93, 6.95)"
